In [1]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, top)
# sys.path

In [2]:
from apps.book.classes.bbooks import BOOKS

In [9]:
init = {'bookid': '0010550825'}
book = BOOKS(**init)

In [11]:
# book.update_errcnt = 0
await book.update_info()
# book.__dict__

update_duration = 1.6947746276855469,final_proxy=None
update_errcnt = 0


In [13]:
book.info

{'store': 'BOOKS',
 'bookid': '0010550825',
 'isbn10': None,
 'isbn13': '9789866000263',
 'title': '惡魔呢喃而來',
 'title2': 'IL SUGGERITORE',
 'author': '作者：多那托．卡瑞西/原文作者：Donato Carrisi/譯者：吳宗璘',
 'publisher': '春天出版社',
 'pub_dt': '2012-07-03',
 'lang': '繁體中文',
 'price_list': 380,
 'price_sale': 300,
 'spec': '平裝/448頁/15x21x2.24cm/普通級/單色印刷/初版',
 'intro': '\n<p align="center"><strong><font color="#ff0000">橫掃英、美、德、法、義大利暢銷榜\xa0 <br/>繼《龍紋身的女孩》後又一席捲全歐書市大作<br/>義大利最具話題性的國際級暢銷小說</font></strong></p>\n<p><strong><font color="#3333ff">\u3000\u3000●已售出美、英、法、德、日、韓、以色列、荷蘭、土耳其、巴西、俄羅斯、希臘、西班牙、葡萄牙、丹麥、中國大陸、塞爾維亞、越南、保加利亞、匈牙利……等多國版權</font></strong></p>\n<p><strong><font color="#3333ff">\u3000\u3000●於英國上市期間供不應求，亞馬遜書店缺貨長達一個多月！</font></strong></p>\n<p><strong><font color="#3333ff">\u3000\u3000●情節錯綜複雜，The Bookseller.com網站譽為︰有史以來最難猜測結局的小說之一。</font></strong></p>\n<p><strong><font color="#3333ff">\u3000\u3000●巴西版出版社重金製作宣傳影片，並買下電影〈隔離島〉播映前廣告強打</font></strong></p>\n<p>\u3000\u3000擁有完美生活和正當職業的商務人士柏曼，被一名神秘男子襲擊而失去知覺，當他醒來時，後車廂裡

In [ ]:
ipscfg.ips_cycle

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
    "Referer": "https://www.books.com.tw/products/0010550825",

}
url = 'https://www.books.com.tw/products/0010550825?sloc=main'
url2 = 'https://www.books.com.tw/product_show/getCommentAjax/0010550825:1:A:M201101_0_getCommentData_P00a400020068:getCommentAjax:M201101_078_view/M201101_078_view'
#
connector = aiohttp.TCPConnector(ssl=True)
TO = aiohttp.ClientTimeout(total=11)
async with aiohttp.ClientSession(connector=connector, timeout=TO) as session:
    async with session.get(url, headers=headers) as r:
        status = r.status
        rtext = await r.text(encoding='utf8')
    async with session.get(url2, headers=headers2) as r:
        status = r.status
        rtext2 = await r.text(encoding='utf8')

In [ ]:
status

In [ ]:
# rtext2
a = '123{}'
a.format(456)

In [ ]:
class INFO:

    def __setattr__(self, name, value):
        self.__dict__[name] = value

In [ ]:
a = INFO()

In [ ]:
a.e = 1

In [ ]:
a.__dict__

In [ ]:
b.test

In [ ]:
object.__setattr__(b, 'test', 101)

In [ ]:
object.__getattribute__(b, 'test')